# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import sys
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings 
warnings.filterwarnings('ignore')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import pickle
import sqlite3

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('ETL_processed_data', engine)

# take a look of df
df.head()

In [ ]:
df[['message','original','genre']].dtypes

In [ ]:
# define the columns "message" + "original" + "genre" as predictors and rest 36 categories as target
X = df.ix[:,1:4]
Y = df.ix[:,4:]

In [ ]:
X['message'][26187]

In [ ]:
X['original'].isnull().sum()/df.shape[0]

In [ ]:
X['genre'].dtype

In [ ]:
Y

In [ ]:
for y in Y.columns.values.tolist():
    print(y)
    print(Y[y].value_counts())
    print('\n')

### Do some data exploratory

In [ ]:
# Take a look of predictors and targets
X.head()

In [ ]:
Y.head()

In [ ]:
# check the null values distribution
X.isnull().sum()

In [ ]:
Y.isnull().sum()

In [ ]:
X['genre'].value_counts()


In [ ]:
X.select_dtypes(include='object')

** Findings
For predictors, "message" and "original" are text variables. "original" has 16,046 missing values. "genre" is 
categorical variable and has 3 levels. I am going to do some preprocessing to deal with missing values and text transform.
Create dummy variables for the categorical one.
For targets, we have no missing values and each label is composed of 0 or 1.

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    
    # Normailization -- lower case + remove puntuation 
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    
    # tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    words = [word for word in tokens if word not in stopwords.words("english")]
    
    # stemmization
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]
        
    return text_lems

In [ ]:
tokenize(X['message'].values[20])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 50, max_depth = 3, min_samples_split = 2)))
])

In [ ]:
# Build text categorical data type selector

class Cate_Text_Selector(BaseEstimator, TransformerMixin):
    
    def __init__(self, dtype):
        self.dtype = dtype
        
    def fit(self, X, y = None):
        return self
    
    def text_selector(self, X):
        
        text_col = []
        cate_col = []
        
        for col in X.select_dtypes(include='object').columns.tolist():
            if len(X[col].unique()) > 0.5 * len(X[X[col].notnull()]):
                text_col.append(col)
            else:
                cate_col.append(col)
                
        return text_col, cate_col
        
    def transform(self, X):
        
        text_col, cate_col = self.text_selector(X)
        
        if self.dtype == 'text':
            return X[text_col].values[:,0]
        if self.dtype == 'category':
            return X[cate_col].values

In [ ]:
cts = Cate_Text_Selector('text')

In [ ]:
cts.fit_transform(df[['message']])#[:,0]##.values

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
# Split train and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X[['message','genre']],Y,test_size = 0.25, random_state = 42)

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_features', Pipeline([
            ('selector',Cate_Text_Selector('text')),
            ('vect', CountVectorizer(tokenizer = tokenize)),
            ('tfidf', TfidfTransformer())
        ])),# Text preprocessing ends
        
        ('cate_features', Pipeline([
            ('selector',Cate_Text_Selector('category')),
            ('dummy', OneHotEncoder())
        ])) # Normal categorical variable preprocessing ends
    ])), # Feature part ends
    
    ('clf', MultiOutputClassifier(SGDClassifier()))
])

In [ ]:
# Train pipeline
pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# get prediction on test dataset
Y_pred = pipeline.predict(X_test)

# Output evaluation score for each target column
target_columns = Y.columns.values

weighted_fscore = 0
for i in np.arange(0,36,1):
    print("Target:{}".format(target_columns[i]))
    print("\n")
    print(classification_report(Y_test.values[i],Y_pred[i]))
    print('\n')
    print('\n')
    weighted_fscore += f1_score(Y_test.values[i],Y_pred[i], average='weighted')

print(weighted_fscore/(i+1))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [20, 50],
    'clf__estimator__max_depth': [3, 6],
    'clf__estimator__min_samples_split': [2,4]
}

cv = GridSearchCV(pipeline, param_grid= parameters)
cv.fit(X_train, Y_train)
print(cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_pred = cv.predict(X_test)

weighted_fscore = 0
for i in np.arange(0,36,1):
    print("Target:{}".format(target_columns[i]))
    print("\n")
    print(classification_report(Y_test[i],cv_pred[i]))
    print('\n')
    print('\n')
    weighted_fscore += f1_score(Y_test[i],cv_pred[i], average='weighted')

print(weighted_fscore/(i+1))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

*** Need to do in pipeline:
    1. data type selector -- class
    2. text variable selector
    3. deal with each data type: 
        1) numerical -- scaler + imputer (mean)
        2) categorical -- imputer ('NaN') + CountVectorizer + tfidf
    4. clf
    5. Put all together
    
*** Need to do in different algorithm
    1. Try SGDClassifier
    2. Try Logistic Regression
    
*** Compile
    1. load data
    2. functions
    3. pipeline
    4. output evaluation results
    4. main()

In [ ]:
#### Add more features -- original, genre


In [ ]:
# Try SGD
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SGDClassifier()))
])

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
def load_data(database_filepath):
    """
    """
    # load data from database
    engine = create_engine(database_filepath)
    df = pd.read_sql_table('ETL_processed_data', engine)
    
    # split training and testing dataset
    X = df.ix[:,1:4]
    Y = df.ix[:,4:]
    
    # get target variable names
    category_names = Y.columns.tolist()
    
    return X, Y, category_names


def tokenize(text):
    """
    """
    # Normailization -- lower case + remove puntuation 
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    
    # tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    words = [word for word in tokens if word not in stopwords.words("english")]
    
    # stemmization
    text_lems = [WordNetLemmatizer().lemmatize(lem).strip() for lem in words]
        
    return text_lems


# Build text categorical data type selector

class Cate_Text_Selector(BaseEstimator, TransformerMixin):
    
    def __init__(self, dtype):
        self.dtype = dtype
        
    def fit(self, X, y = None):
        return self
    
    def text_selector(self, X):
        
        text_col = []
        cate_col = []
        
        for col in X.select_dtypes(include='object').columns.tolist():
            if len(X[col].unique()) > 0.5 * len(X[X[col].notnull()]):
                text_col.append(col)
            else:
                cate_col.append(col)
                
        return text_col, cate_col
        
    def transform(self, X):
        
        text_col, cate_col = self.text_selector(X)
        
        if self.dtype == 'text':
            return X[text_col].values[:,0]
        if self.dtype == 'category':
            return X[cate_col].values


def build_model():
    """
    """
    
    # feature preprocessing pipeline
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_features', Pipeline([
                ('selector',Cate_Text_Selector('text')),
                ('vect', CountVectorizer(tokenizer = tokenize)),
                ('tfidf', TfidfTransformer())
            ])),# Text preprocessing ends

            ('cate_features', Pipeline([
                ('selector',Cate_Text_Selector('category')),
                ('dummy', OneHotEncoder())
            ])) # Normal categorical variable preprocessing ends
        ])), # Feature part ends

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    # grid search
    parameters = {
        #'clf__estimator__n_estimators': [20, 50],
        #'clf__estimator__max_depth': [3, 6]
        'clf__estimator__min_samples_split': [2,4]
        #'clf__estimator__loss': ['log', 'hinge']
        #'clf__estimator__penalty': ['l2']
        #'clf__estimator__alpha': [0.001, 0.0001]
    }

    cv = GridSearchCV(pipeline, param_grid= parameters)
    
    return cv


def evaluate_model(model, X_test, Y_test, category_names):
    """
    """
    # prediction on test dataset
    Y_pred = model.predict(X_test)
    
    # initialize aggregated avg score
    weighted_fscore = 0
    
    for i in np.arange(0,36,1):
        print("Target:{}".format(category_names[i]))
        print("\n")
        print(classification_report(Y_test.values[i],Y_pred[i]))
        print('\n')
        print('\n')
        weighted_fscore += f1_score(Y_test.values[i],Y_pred[i], average='weighted')

    print("Overall average f1 score of all categories are: {}".format(weighted_fscore/(i+1)))
    

def save_model(model, model_filepath):
    """Save trained model to sepcified file path
    """
    pickle.dump(model,model_filepath)

    
def main():
    
    X, Y, category_names = load_data('sqlite:///DisasterResponse.db')
    X_train, X_test, Y_train, Y_test = train_test_split(X[['message','genre']], Y, test_size=0.2)

    print('Building model...')
    model = build_model()

    print('Training model...')
    model.fit(X_train, Y_train)

    print('Evaluating model...')
    evaluate_model(model, X_test, Y_test, category_names)
    

    print('Saving model...\n    MODEL: {}'.format('Random_Forest_trained.pickle'))
    save_model(model, 'Random_Forest_trained.pickle')

    #print('Trained model saved!')
    
main()

Building model...
Training model...


NameError: name 'model' is not defined

In [43]:
engine = create_engine('sqlite:///data/DisasterResponse.db')

In [44]:
engine

Engine(sqlite:///data/DisasterResponse.db)

In [40]:
engine = create_engine('sqlite:///{}'.format('data/DisasterResponse.db'))

In [42]:
engine

Engine(sqlite:///.data/DisasterResponse.db)

In [41]:
df = pd.read_sql_table('ETL_processed_data', engine)

OperationalError: (sqlite3.OperationalError) unable to open database file (Background on this error at: http://sqlalche.me/e/e3q8)

In [17]:
conn = sqlite3.connect('../data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM ETL_processed_data', conn)

OperationalError: unable to open database file

In [7]:
df

,id,message,original,genre,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
